Airline AI Assistant - this is an extension of a previous project "Airline Agent". 
- In this notebook I will be creating another Agent to Translate all the responses into a different language.
- Adding more tools to handle booking a flight

In [184]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

Load in the API keys

In [185]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


System Prompts

In [236]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "If a flight is not available on a certain date, say that it is not available."
# system_message += "If a flight is available on a certain date, and the customer requests to book it, say here is your ticket, have a nice flight."

# This last system message is a good way to ensure the agent does not hallucinate

Forcing dark mode in Gradio

In [274]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

#### Tools
- Tools are an incredibly powerful feature provided by the frontier LLMs.
- With tools, you can write a function, and have the LLM call that function as part of its response.
- Sounds almost spooky.. we're giving it the power to run code on our machine? Well, kinda.

In [275]:
ticket_prices = {
    "london": "$799", 
    "paris": "$899", 
    "tokyo": "$1400", 
    "berlin": "$499"
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [276]:
ticket_availability = {
    "london": ["11th", "12th"],
    "paris": ["13th", "14th"],
    "tokyo": ["15th", "16th"],
    "berlin": ["13th", "14th"]
}


def get_ticket_availability(destination_city):
    print(f"Tool get_ticket_dates called for {destination_city}")
    city = destination_city.lower()
    return ticket_availability.get(city, "Unknown")

In [277]:
# import random

# def book_flight(destination_city):
#     """
#     Simulates booking a flight and returns a confirmation number.
#     """
#     city = destination_city.lower()
#     print(f"Attempting to book flight for {destination_city} on {get_ticket_availability(destination_city)} for the price of {get_ticket_price(destination_city)}.")
    
#     # # Simulating a booking process (Replace with real API calls if needed)
#     # if not destination_city or not get_ticket_availability(destination_city):
#     #     return {"status": "error", "message": "There is no flight available on that date."}
    
#     confirmation_number = f"FLIGHT-{random.randint(100000, 999999)}"
#     return {
#         "status": "success",
#         "confirmation_number": confirmation_number,
#         "destination_city": destination_city,
#         "ticket_availability": get_ticket_availability(destination_city),
#         "ticket_price": get_ticket_price(destination_city)
#     }

In [278]:
import random

def book_flight(destination_city):
    print(f"Tool to book a flight for {destination_city}")
    city = destination_city.lower()
    availability = ticket_availability.get(city, "Unknown")
    price =ticket_prices.get(city, "Unknown")
    
    confirmation_number = f"FLIGHT-{random.randint(100000, 999999)}"
    
    return {
        "status": "success",
        "confirmation_number": confirmation_number,
        "destination_city": destination_city,
        "ticket_availability": availability,
        "ticket_price": price
    }

In [279]:
book_flight("Berlin")

Tool to book a flight for Berlin


{'status': 'success',
 'confirmation_number': 'FLIGHT-992700',
 'destination_city': 'Berlin',
 'ticket_availability': ['13th', '14th'],
 'ticket_price': '$499'}

In [280]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [281]:
get_ticket_availability("Berlin")

Tool get_ticket_dates called for Berlin


['13th', '14th']

In [282]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [283]:
availability_function = {
    "name": "get_ticket_availability",
    "description": "Call this whenever you need to know the date of a flight to a particular city, for example when a customer asks 'Are there any available flights to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
book_flight_function = {
    "name": "book_flight",
    "description": "Call this whenever you need to book a flight for a customer, for example when a customer says 'I would like to book a flight to this city on this date.'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city","availability"],
        "additionalProperties": False
    }
}

In [305]:
tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": availability_function},
         {"type": "function", "function": book_flight_function}]

#### Getting OpenAI to use our Tool
- There's some fiddly stuff to allow OpenAI "to call our tool"
- What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.
- Here's how the new chat function looks:

In [313]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city
    
    elif tool_call.function.name == "get_ticket_availability":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        availability = get_ticket_availability(city)  
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "availability": availability}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city
    
    elif tool_call.function.name == "book_flight":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        availability = get_ticket_availability(city)
        price = get_ticket_price(city) 
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "availability": availability, 
                                   "ticket_price" : price}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response, city, availability, price


#### Chat Function
- we have added in two new message lines to allow our agent to use this tool
    - adding in print statments could make this more clear

- messages = response.choices[0].message 
    - This is nothing more than what we get back from our Assistant asking to run a tool
- response, city = handle_tool_call(message)
    - This is the result of calling the tool function

In [307]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message 
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [308]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

# Interesting, I get an error when I don't specify the city name when asking for a price

# Interesting, I get an error when asking to book a flight even after being told the availability
# I also receive an error when asking to the book a flight to Berlin on the 13th.

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


Talker Function

In [309]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

In [310]:
# def chat(history):
#     messages = [{"role": "system", "content": system_message}] + history
#     response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
#     image = None
    
#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         image = artist(city)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
        
#     reply = response.choices[0].message.content
#     history += [{"role":"assistant", "content":reply}]

#     # Comment out or delete the next line if you'd rather skip Audio for now..
#     talker(reply)
    
#     return history, image

In [311]:
# gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

In [ ]:
# # More involved Gradio code as we're not using the preset Chat interface!
# # Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

# with gr.Blocks(js=force_dark_mode) as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500, type="messages")
#         image_output = gr.Image(height=500)
#     with gr.Row():
#         entry = gr.Textbox(label="Chat with our AI Assistant:")
#     with gr.Row():
#         clear = gr.Button("Clear")

#     def do_entry(message, history):
#         history += [{"role":"user", "content":message}]
#         return "", history

#     entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
#         chat, inputs=chatbot, outputs=[chatbot, image_output]
#     )
#     clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch(inbrowser=True)

Tool get_ticket_dates called for Berlin
